# ATEK Data Proprocessing Example (EFM model)

### 1. Load the example adt data

In [1]:
import os

from atek.data_preprocess.sample_builders.efm_sample_builder import EfmSampleBuilder
from atek.data_preprocess.wds_writer import AtekWdsWriter

from omegaconf import OmegaConf

import logging
from logging import StreamHandler

handler = StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

logger = logging.getLogger()
logger.addHandler(handler)

example_adt_data_dir = "/home/louy/Calibration_data_link/Atek/2024_05_07_EfmDataTest/adt_data_example/Apartment_release_clean_seq134/1WM103600M1292_optitrack_release_clean_seq134"
example_ase_data_dir = "/home/louy/Calibration_data_link/Atek/2024_05_07_EfmDataTest/ase_data_example/euston_simulation_100077_device0"
config_path = os.path.join("/home/louy/Calibration_data_link/Atek/2024_05_07_EfmDataTest/efm_euston_medium_conf_for_atek.yaml")

conf = OmegaConf.load(config_path)

### 2. Sample builder example

In [8]:
sample_builder = EfmSampleBuilder(
    conf=conf.processors,
    vrs_file=os.path.join(example_ase_data_dir, "video.vrs"),
    mps_files={
        "mps_closedloop_traj_file": os.path.join(
            example_ase_data_dir, "aria_trajectory.csv"),
        "mps_semidense_points_file": os.path.join(example_ase_data_dir, "maps", "maps_v1", "globalcloud_GT.csv.gz"),
        "mps_semidense_observations_file": os.path.join(example_ase_data_dir, "maps", "maps_v1", "observations.csv.gz"),
    },
    gt_files={
        "obb3_file": os.path.join(example_ase_data_dir, "3d_bounding_box.csv"),
        "obb3_traj_file": os.path.join(
            example_ase_data_dir, "scene_objects.csv"
        ),
        "instance_json_file": os.path.join(
            example_ase_data_dir, "instances.json"
        ),
    },
    depth_vrs_file = os.path.join(example_ase_data_dir, "depth_images.vrs")
)

"""
sample_builder = EfmSampleBuilder(
    conf=conf.processors,
    vrs_file=os.path.join(example_adt_data_dir, "video.vrs"),
    mps_files={
        "mps_closedloop_traj_file": os.path.join(
            example_adt_data_dir, "aria_trajectory.csv"),
        "mps_semidense_points_file": os.path.join(example_adt_data_dir, "mps", "slam", "semidense_points.csv.gz"),
        "mps_semidense_observations_file": os.path.join(example_adt_data_dir, "mps", "slam", "semidense_observations.csv.gz"),
    },
    gt_files={
        "obb3_file": os.path.join(example_adt_data_dir, "3d_bounding_box.csv"),
        "obb3_traj_file": os.path.join(
            example_adt_data_dir, "scene_objects.csv"
        ),
        "instance_json_file": os.path.join(
            example_adt_data_dir, "instances.json"
        ),
    },
    depth_vrs_file = os.path.join(example_adt_data_dir, "depth_images.vrs")
)
"""



[MultiRecordFileReader][DEBUG]: Opened file '/home/louy/Calibration_data_link/Atek/2024_05_07_EfmDataTest/ase_data_example/euston_simulation_100077_device0/video.vrs' and assigned to reader #0
2024-05-21 16:57:38,193 - INFO - loading global semi-dense points from /home/louy/Calibration_data_link/Atek/2024_05_07_EfmDataTest/ase_data_example/euston_simulation_100077_device0/maps/maps_v1/globalcloud_GT.csv.gz
[VrsDataProvider][INFO]: streamId 214-1/camera-rgb activated
[VrsDataProvider][INFO]: streamId 1201-1/camera-slam-left activated
[VrsDataProvider][INFO]: streamId 1201-2/camera-slam-right activated
[VrsDataProvider][INFO]: streamId 1202-1/imu-right activated
[VrsDataProvider][INFO]: streamId 1202-2/imu-left activated
[MultiRecordFileReader][DEBUG]: Opened file '/home/louy/Calibration_data_link/Atek/2024_05_07_EfmDataTest/ase_data_example/euston_simulation_100077_device0/video.vrs' and assigned to reader #0
[VrsDataProvider][INFO]: streamId 214-1/camera-rgb activated
[VrsDataProvider]

'\nsample_builder = EfmSampleBuilder(\n    conf=conf.processors,\n    vrs_file=os.path.join(example_adt_data_dir, "video.vrs"),\n    mps_files={\n        "mps_closedloop_traj_file": os.path.join(\n            example_adt_data_dir, "aria_trajectory.csv"),\n        "mps_semidense_points_file": os.path.join(example_adt_data_dir, "mps", "slam", "semidense_points.csv.gz"),\n        "mps_semidense_observations_file": os.path.join(example_adt_data_dir, "mps", "slam", "semidense_observations.csv.gz"),\n    },\n    gt_files={\n        "obb3_file": os.path.join(example_adt_data_dir, "3d_bounding_box.csv"),\n        "obb3_traj_file": os.path.join(\n            example_adt_data_dir, "scene_objects.csv"\n        ),\n        "instance_json_file": os.path.join(\n            example_adt_data_dir, "instances.json"\n        ),\n    },\n    depth_vrs_file = os.path.join(example_adt_data_dir, "depth_images.vrs")\n)\n'

In [9]:
# Print the frame dataclass keys
sample = sample_builder.get_sample_by_timestamps_ns([100_000_000])
from dataclasses import asdict
print(asdict(sample).keys())

Loaded #closed loop trajectory poses records: 210
dict_keys(['camera_rgb', 'camera_slam_left', 'camera_slam_right', 'mps_traj_data', 'mps_semidense_point_data', 'camera_rgb_depth', 'gt_data'])


### 2. Write wds files

In [10]:
from atek.data_preprocess.wds_writer import (
    AtekWdsWriter,
)
from atek.data_preprocess.subsampling_lib.temporal_subsampler import CameraTemporalSubsampler

subsampler = CameraTemporalSubsampler(vrs_file = os.path.join(example_ase_data_dir, "video.vrs"), conf = conf.camera_temporal_subsampler)

conf.wds_writer.prefix_string = "test"
atek_wds_writer = AtekWdsWriter(output_path = "/home/louy/Calibration_data_link/Atek/2024_05_07_EfmDataTest/wds_output/test_ase_1", conf = conf.wds_writer)

for i in range(subsampler.get_total_num_samples()):
    timestamps_ns = subsampler.get_timestamps_by_sample_index(i)

    sample = sample_builder.get_sample_by_timestamps_ns(timestamps_ns)
    if sample is not None:
        atek_wds_writer.add_sample(data_sample = sample)

atek_wds_writer.close()


[MultiRecordFileReader][DEBUG]: Opened file '/home/louy/Calibration_data_link/Atek/2024_05_07_EfmDataTest/ase_data_example/euston_simulation_100077_device0/video.vrs' and assigned to reader #0
[VrsDataProvider][INFO]: streamId 214-1/camera-rgb activated
[VrsDataProvider][INFO]: streamId 1201-1/camera-slam-left activated
[VrsDataProvider][INFO]: streamId 1201-2/camera-slam-right activated
[VrsDataProvider][INFO]: streamId 1202-1/imu-right activated
[VrsDataProvider][INFO]: streamId 1202-2/imu-left activated


TypeError: 'float' object is not iterable